In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load CRF data
df = pd.read_csv('../data/crf_simulated.csv')

# Convert symptom notes to numeric volatility score
symptom_map = {'Stable': 0, 'Improved': 1, 'Worsened': 2}
df['Symptom_Code'] = df['Symptom_Notes'].map(symptom_map)

# Group by Patient_ID to create patient-level features
agg_df = df.groupby('Patient_ID').agg({
    'RA_Severity_Score': ['mean', lambda x: x.iloc[-1] - x.iloc[0]],
    'Symptom_Code': lambda x: x.nunique(),
    'Drug_Dosage_mg': lambda x: x.iloc[-1],
    'Consent_Status': lambda x: 1 if x.iloc[0] == 'Yes' else 0
})

# Rename columns
agg_df.columns = ['Severity_Mean', 'Severity_Trend', 'Symptom_Volatility', 'Final_Dosage', 'Consent_Binary']
agg_df.reset_index(inplace=True)

# Define features and target
X = agg_df.drop(columns=['Patient_ID', 'Final_Dosage'])
y = agg_df['Final_Dosage']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("📊 Patient-Level Classification Report:")
print(classification_report(y_test, y_pred, zero_division=0))


📊 Patient-Level Classification Report:
              precision    recall  f1-score   support

          10       0.25      0.33      0.29         3
          20       0.36      0.57      0.44         7
          30       0.40      0.20      0.27        10

    accuracy                           0.35        20
   macro avg       0.34      0.37      0.33        20
weighted avg       0.36      0.35      0.33        20



**Modeling Notes:**  
The patient-level model shows moderate performance, with Class 20 predicted most accurately. Class imbalance and limited features contribute to low precision and recall in other classes. Future iterations will expand feature engineering and explore alternate targets such as flare-up risk or symptom progression. All data is simulated and documented for ethics compliance.
